# Football Data Analysis of Italian Serie A

The data is taken from https://datahub.io/sports-data/italian-serie-a#resource-italian-serie-a_zip

## Imports

In [ ]:
from pyspark import SparkContext,  SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# For the purposes of debugging

%load_ext autoreload
%autoreload 2

## Load Data

We shall load only the data for the 2018-2019 season, as the data for the other seasons is not complete and is too big to work with.
For the purposes of this notebook, it will suffice.

In [ ]:
# Create Spark Context

conf = SparkConf().setAppName("Spark").setMaster("local[*]")
# Create context if it doesn't exist
try:
    sc = SparkContext(conf=conf)
except:
    pass

# Create Spark Session

spark = SparkSession.builder.appName("Spark").getOrCreate()

In [ ]:
# Load the data from the csv file 

original_fb_data = spark.read.csv("./data/season-1819.csv", header=True, inferSchema=True)


In [ ]:

# Print the schema of the dataframe

original_fb_data.printSchema()

### Exercise 2 

Eliminate the columns that are not needed for the analysis - ['Div', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA']

In [ ]:
# Eliminate the columns that are not needed

columns_to_drop = ['Div', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA']
fb_data = original_fb_data.drop(*columns_to_drop)

# Print the schema of the dataframe

# fb_data.printSchema()


### Exercise 3

Create a columns.txt file that contains a map of the columns to their new names. Use this file to rename the columns.


In [ ]:
# Create columns.txt file - columns FTHG, FTAG, FTR, HTHG, HTAG, HTR, HS, AS, HST, AST, HF, AF, HC, AC, HY, AY, HR, AR  

columns_to_rename = ['FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']

# Create a dictionary to the new names 

new_names = {'FTHG':'final_time_home_goals', 'FTAG':'final_time_away_goals', 'FTR':'final_time_result', 'HTHG':'half_time_home_goals', 
             'HTAG':'half_time_away_goals', 'HTR':'half_time_result', 'HS':'home_shots', 'AS':'away_shots', 'HST':'home_shots_on_target',
               'AST':'away_shots_on_target', 'HF':'home_fouls', 'AF':'away_fouls', 'HC':'home_corners', 'AC':'away_corners', 'HY':'home_yellow_cards', 
               'AY':'away_yellow_cards', 'HR':'home_red_cards', 'AR':'away_red_cards'}

# Save that dictionary to a file

from utils import create_columns

create_columns(new_names, "./data")









In [ ]:
# Load map from columns.txt file

column_map = {}

with open("./data/columns.txt", "r") as f:
    for line in f.readlines():
        line = line.strip()
        key, value = line.split("->")
        column_map[key.strip()] = value.strip()

# Rename some of the columns from the dictionary and keep the rest

fb_data_renamed = fb_data.select([col(c).alias(column_map.get(c, c)) for c in fb_data.columns])

# Print the schema of the dataframe

# fb_data_renamed.printSchema()

### Exercise 4

Change final_time_result to categories home, draw, away

In [ ]:
# Change the values in the final_time_result column to home, draw, away

fb_data_renamed = fb_data_renamed.withColumn("final_time_result", when(col("final_time_result") == "H", "home")
                                             .when(col("final_time_result") == "D", "draw")
                                             .otherwise("away"))

### Exercise 5

Find all the fames where the home team won

In [ ]:
# Find in what games the home team won

home_wins = fb_data_renamed.filter(col("final_time_result") == "home")

### Exercise 6 

Convert the date column to a "yyyy-MM-dd" format

In [ ]:
# Convert the date column to a date type

fb_data_renamed = fb_data_renamed.withColumn("Date", to_date(col("Date"), "dd/MM/yyyy"))

# Show the first 5 rows of the date column

# fb_data_renamed.select("Date").show(5)


### Exercise 7

Add precision column to all the games where group with the upper goals has lost

In [ ]:

# Find all the games where the team with the upper shots on target has lost

upper_shots_lost = fb_data_renamed.filter((col("home_shots_on_target") > col("away_shots_on_target")) & (col("final_time_result") == "away") \
| (col("home_shots_on_target") < col("away_shots_on_target")) & (col("final_time_result") == "home"))


upper_shots_lost = upper_shots_lost.withColumn("home_precision", col("home_shots_on_target") / col("home_shots")) \
.withColumn("away_precision", col("away_shots_on_target") / col("away_shots"))

# Show the first 5 rows of the teams and the precision columns

upper_shots_lost.select("HomeTeam", "AwayTeam", "home_precision", "away_precision").show(5)


### Exercise 8

Find the pure winning games where the winning team has no opposite goals


In [ ]:
# Find games where the winning team has no opposite goals

pure_wins = fb_data_renamed.filter((col("final_time_result") == "home") & (col("final_time_away_goals") == 0) | (col("final_time_result") == "away") & (col("final_time_home_goals") == 0))

# Show the first 5 rows of the teams and the goals columns

pure_wins.select("HomeTeam", "AwayTeam", "final_time_home_goals", "final_time_away_goals").show(5)

### Exercise 9

Find the game with the most total goals

In [ ]:
# Find the highest total goals scored in a game

fb_with_total_goals = fb_data_renamed.withColumn("total_goals", col("final_time_home_goals") + col("final_time_away_goals"))
highest_goals = fb_with_total_goals.sort(desc("total_goals"))

# Get all the games where the total goals are equal to the highest total goals

highest_goals = highest_goals.filter(col("total_goals") == highest_goals.select("total_goals").first()[0])

# Show the first 5 rows of the teams and the total goals columns

highest_goals.select("Date", "HomeTeam", "AwayTeam", "total_goals").show(5)

### Exercise 10

Add an interest rate column - a game considered interest rate high/medium/low if:
- High if more the 5 goals conquered, or the game is between the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio]
- Medium if the number of goals conquered is between 3 and 5, or the game is between at least one of the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio]
- Low if the number of goals conquered is less than 3, and the game is not between any of the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio] 


In [ ]:
# Add a column of interest rate based on

# a game considered interest rate high/medium/low if:
# - High if more the 5 goals conquered, or the game is between the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio]
# - Medium if the number of goals conquered is between 3 and 5, or the game is between at least one of the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio]
# - Low if the number of goals conquered is less than 3, and the game is not between any of the teams [Juventus, Inter, Milan, Roma, Napoli, Lazio] 

fb_data_renamed = fb_with_total_goals.withColumn("interest_rate", when((col("total_goals") > 5) | ((col("HomeTeam") == "Juventus") | (col("HomeTeam") == "Inter") | (col("HomeTeam") == "Milan") | (col("HomeTeam") == "Roma") | (col("HomeTeam") == "Napoli") | (col("HomeTeam") == "Lazio")) | ((col("AwayTeam") == "Juventus") | (col("AwayTeam") == "Inter") | (col("AwayTeam") == "Milan") | (col("AwayTeam") == "Roma") | (col("AwayTeam") == "Napoli") | (col("AwayTeam") == "Lazio")), "high")
                                                .when(((col("total_goals") >= 3) & (col("total_goals") <= 5)) | ((col("HomeTeam") == "Juventus") | (col("HomeTeam") == "Inter") | (col("HomeTeam") == "Milan") | (col("HomeTeam") == "Roma") | (col("HomeTeam") == "Napoli") | (col("HomeTeam") == "Lazio")) | ((col("AwayTeam") == "Juventus") | (col("AwayTeam") == "Inter") | (col("AwayTeam") == "Milan") | (col("AwayTeam") == "Roma") | (col("AwayTeam") == "Napoli") | (col("AwayTeam") == "Lazio")), "medium")              
                                                .otherwise("low"))

### Exercise 11

Out of the defined interesting teams, find the team that has lost the most interesting games

In [ ]:
# Out of the defined interesting teams, find the team that has lost the most interesting games

interesting_teams = ["Juventus", "Inter", "Milan", "Roma", "Napoli", "Lazio"]

# Find the games where the interesting teams have lost - either in home or away

interesting_teams_lost = fb_data_renamed.filter((col("final_time_result") == "away") & (col("AwayTeam").isin(interesting_teams)) | (col("final_time_result") == "home") & (col("HomeTeam").isin(interesting_teams)))

# Find the team that has lost the most interesting games

interesting_teams_lost.groupBy("HomeTeam").count().sort(desc("count")).show(1)




### Exercise 12

Find the most aggressive team - the team that got the most yellow/red cards
Weight the red card as 2 yellow cards


In [ ]:
# Transform the home/away teams to a dataframe grouping teams containing for each team the sum of all the quantities

def remove_sum_home_away(col_name):
    return col_name.replace("sum(", "").replace(")", "").replace("home_", "").replace("away_", "")

# Columns with home or away in the name

quantities_team_columns = [col(column).alias(remove_sum_home_away(column)) for column in fb_data_renamed.columns if "home" in column or "away" in column]

# Create a dataframe with the home teams

home_teams = fb_data_renamed.select(col("HomeTeam").alias("Team"), "final_time_home_goals", "final_time_away_goals", "home_shots", "home_shots_on_target", "home_fouls", "home_corners", \
 "home_yellow_cards", "home_red_cards").groupby("Team").sum()

# Columns with home or away in the name

quantities_team_columns = [col(column).alias(remove_sum_home_away(column)) for column in home_teams.columns if "home" in column]

home_teams = home_teams.select("Team", *quantities_team_columns)

# Create a dataframe with the away teams

away_teams = fb_data_renamed.select(col("AwayTeam").alias("Team"), "final_time_home_goals", "final_time_away_goals", "away_shots", "away_shots_on_target", "away_fouls", "away_corners", \
    "away_yellow_cards", "away_red_cards").groupby("Team").sum()

# Columns with home or away in the name

quantities_team_columns = [col(column).alias(remove_sum_home_away(column)) for column in away_teams.columns if "away" in column]

away_teams = away_teams.select("Team", *quantities_team_columns)

# Join the home and away teams and sum the columns
# TODO: need to join and sum up the values


teams = home_teams.join(away_teams, "Team").select("Team", *[sum(column).alias(column) for column in quantities_team_columns])

# Show the first 5 rows of the teams dataframe

teams.show(5)







### Exercise 13 

We have the columns of the agencies that give the odds for the games. We want to find the agency that has the best odds for the home team winning.


In [ ]:
# Agency columns start with the agency name and end with the result of the game - home/away/draw
# e.g, The agency "B365" has the columns "B365H", "B365D", "B365A" - home win, draw, away win

# For each game, find the agency that has the highest odds for the home team to win

# Extract agencies names

agencies = [column.split("_")[0] for column in fb_data_renamed.columns if column.endswith("H")]

# Create a dataframe with the odds for the home team to win

home_wins_odds = fb_data_renamed.select("HomeTeam", "B365H", "BWH", "IWH", "LBH", "PSH", "WHH", "SJH", "VCH", "GBH", "BSH").withColumnRenamed("HomeTeam", "Team").withColumnRenamed("B365H", "B365").withColumnRenamed("BWH", "BW").withColumnRenamed("IWH", "IW").withColumnRenamed("LBH", "LB").withColumnRenamed("PSH", "PS").withColumnRenamed("WHH", "WH").withColumnRenamed("SJH", "SJ").withColumnRenamed("VCH", "VC").withColumnRenamed("GBH", "GB").withColumnRenamed("BSH", "BS")

# Find the agency that has the highest odds for the home team to win

home_wins_odds = home_wins_odds.withColumn("max_home_win_odds", greatest(*[column for column in home_wins_odds.columns if column != "Team"]))

# Find the agency that has the achieve the highest odds
conditions = [when(col("max_home_win_odds") == col(agency + "H")) for agency in agencies]
home_wins_odds = home_wins_odds.withColumn("agency_with_max_home_win_odds", *conditions)

# Show the first 5 rows of the home_wins_odds dataframe

home_wins_odds.show(5)



